In [4]:
import glob
import numpy as np
import gymu
from gymu.data import Composable
import gym_pygame
import jnu as J


env = gymu.make("Alone-v0")
iterator = gymu.iterator(env, mode=gymu.mode.sard, max_length=20)
gymu.data.write_episode(iterator, path="Alone-v0/ep-0.tar.gz")
gymu.data.write_episode(iterator, path="Alone-v0/ep-1.tar.gz")

episodes = glob.glob("./Alone-v0/*.tar.gz")
dataset = gymu.data.dataset(episodes).then(Composable.decode())
dataset = dataset.then(gymu.data.Composable.window(window_size=3))

def cat(x):
    x['state'] = np.concatenate([*x['state']], axis=-1)
    return x
dataset = dataset.map(cat)
dataset = dataset.then(gymu.data.Composable.mode(gymu.mode.sa))

J.images([x.state for x in dataset])
    
    





0it [00:00, ?it/s]

0it [00:00, ?it/s]

interactive(children=(IntSlider(value=0, description='x', layout=Layout(width='99%'), max=35), Output()), _dom…

In [ ]:

from gymu.utils import overload

@overload
def f():
    pass

@f.args(int, int)
def f(x, y):
    print('two integers')

@f.args(float)
def f(x):
    print('one float')

    
print(f.cases)
f(1.)
f(1,1)

In [ ]:
from more_itertools import pairwise


iterator = pairwise([1,2,3,4,5])

x = next(iterator)
for x in iterator:
    print(x)


In [2]:
import gymu
import jnu as J
import numpy as np

dataset = gymu.data.dataset(["./NORMAL-300k/ep-0000.tar.gz","./NORMAL-300k/ep-0000.tar.gz"]).then(gymu.data.Composable.decode())
dataset = dataset.then(gymu.data.Composable.mode(gymu.mode.sas))

s = [np.concatenate([x.state, (x.next_state if x.next_state is not None else np.zeros_like(x.state))], axis=-1) for x in dataset]
 


    

FileNotFoundError: [Errno 2] No such file or directory: './NORMAL-300k/ep-0000.tar.gz'

In [ ]:
J.images(s, scale=3)

In [ ]:
import glob
import numpy as np
import gymu
from gymu.data import Composable
import gym_pygame
import jnu as J


env = gymu.make("Alone-v0")
iterator = gymu.iterator(env, mode=gymu.mode.sard, max_length=20)
gymu.data.write_episode(iterator, path="Alone-v0/ep-0.tar.gz")
gymu.data.write_episode(iterator, path="Alone-v0/ep-1.tar.gz")

episodes = glob.glob("./Alone-v0/*.tar.gz")
dataset = gymu.data.dataset(episodes).then(Composable.decode())
dataset = dataset.then(gymu.data.Composable.mode(gymu.mode.sas, ignore_last=False))

s = [np.concatenate([x.state, (x.next_state if x.next_state is not None else np.zeros_like(x.state))], axis=-1) for x in dataset]
J.images(s)      








In [ ]:
import jnu as J
import itertools
import gymu
import glob
from gymu.data import Composable


from torch.utils.data import DataLoader



            
            

episodes = glob.glob("./NORMAL-300k/*.tar.gz")

dataset = gymu.data.dataset(episodes)
dataset = dataset.then(Composable.mode(gymu.mode.sa))
dataset = dataset.shuffle(256).batched(256)
loader = DataLoader(dataset, batch_size=None, num_workers=12, prefetch_factor=16)

for (s, a) in loader:
    print(s.shape, a.shape)